## Crawling Berita

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [9]:
import requests, re, time
from bs4 import BeautifulSoup
import pandas as pd

def get_article_content(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")
    # coba beberapa opsi
    paragraphs = soup.select("div.detail-content p")
    if not paragraphs:
        paragraphs = soup.select("div.article-content p")
    if not paragraphs:
        return ""
    return " ".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

def print_progress(category, page, total_pages):
    print(f"[{category}] Halaman {page}/{total_pages} selesai...")

def berita(categories, pages_per_category=1):
    start_time = time.time()
    BASE_URL = "https://www.idx.co.id/id/berita/{}/?page={}"

    data = {"id_berita": [], "judul_berita": [], "isi_berita": [], "kategori_berita": []}

    for cat in categories:
        for page in range(1, pages_per_category + 1):
            url = BASE_URL.format(cat, page)
            print("Mengambil URL:", url)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            # debug print
            print(soup.prettify()[:1000])

            # beberapa opsi selector berita
            articles = soup.select("div.idx-berita-list a.idx-berita-title")
            if not articles:
                articles = soup.select("ul.idx-list-berita li a")
            if not articles:
                articles = soup.select("div.card-body a")
            if not articles:
                print(f"⚠️ Tidak menemukan elemen artikel untuk kategori {cat} halaman {page}")
                continue

            for a in articles:
                link = a.get("href")
                if not link:
                    continue
                if not link.startswith("http"):
                    link = "https://www.idx.co.id" + link
                title = a.get_text(strip=True)
                id_match = re.search(r"/berita/[^/]+/([^/?#]+)", link)
                berita_id = id_match.group(1) if id_match else None

                try:
                    content_full = get_article_content(link)
                except Exception as e:
                    print(f"⚠️ Error ambil isi berita: {e}")
                    content_full = ""

                data["id_berita"].append(berita_id)
                data["judul_berita"].append(title)
                data["isi_berita"].append(content_full)
                data["kategori_berita"].append(cat)

            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("idx_berita.csv", index=False, encoding="utf-8-sig")

    elapsed = int(time.time() - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")
    return df

categories = ["siaran-pers", "unusual-market-activity-uma", "artikel"]
df = berita(categories, pages_per_category=1)
print(df)


Mengambil URL: https://www.idx.co.id/id/berita/siaran-pers/?page=1
<!DOCTYPE html>
<html lang="en-US">
 <head>
  <title>
   Just a moment...
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="noindex,nofollow" name="robots"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <style>
   *{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-h

In [8]:
print(soup.prettify()[:1000])


NameError: name 'soup' is not defined